In [59]:
import pandas as pd
import numpy as np
import pickle
import time

In [60]:
label_list = [1,2,3,4,5,7]

In [61]:
# Choose if you want to look at EEG or EMG data

data_type = "EEG"
data_type = "EMG"

In [62]:
# choose individuum
subject = "293"

In [63]:
# Load persistence diagrams

persistence_diagrams  = np.load('Embeddings_and_Persistence_Diagrams/'+str(subject)+'/'+str(data_type)+'/Persistence_Diagrams_All_Labels.npy', \
    allow_pickle=True).item() # .item() to convert the dtype to dict again

In [64]:
reshaped_persistence_diagrams = {}

for label in label_list:
    reshaped_persistence_diagrams["Label_"+str(label)] = [persistence_diagram[0] for persistence_diagram in list(persistence_diagrams["Label_"+str(label)])]

persistence_diagrams = reshaped_persistence_diagrams

In [65]:
# Adcock-Carlson coordinates

In [66]:
def carlsson_coordinates(persistence_diagrams):
    n = len(persistence_diagrams)
    persistence_diagrams_features_cc1 = np.zeros(n)
    persistence_diagrams_features_cc2 = np.zeros(n)
    persistence_diagrams_features_cc3 = np.zeros(n)
    persistence_diagrams_features_cc4 = np.zeros(n)
    start = time.time()
    ymax = 0
    for i in range(0,n):
        if len(persistence_diagrams[i])>0:
            b = np.max(persistence_diagrams[i][:,1])
        else:
            b = ymax
        if ymax < b:
            ymax = b
        else:
            ymax = ymax
    for i in range(0,n):
        if len(persistence_diagrams[i])>0:
            x = persistence_diagrams[i][:,0]
            y = persistence_diagrams[i][:,1]
            persistence_diagrams_features_cc1[i] = sum(x*(y-x))
            persistence_diagrams_features_cc2[i] = sum((ymax - y)*(y-x))
            persistence_diagrams_features_cc3[i] = sum(x**2*(y-x)**4)
            persistence_diagrams_features_cc4[i] = sum((ymax-y)**2*(y-x)**4)
        else:
            persistence_diagrams_features_cc1[i] = 0
            persistence_diagrams_features_cc2[i] = 0
            persistence_diagrams_features_cc3[i] = 0
            persistence_diagrams_features_cc4[i] = 0
            
    return persistence_diagrams_features_cc1, persistence_diagrams_features_cc2, persistence_diagrams_features_cc3, persistence_diagrams_features_cc4

In [67]:
def create_dataframe(data_type, X_features_cc1, X_features_cc2, X_features_cc3, X_features_cc4, label, train):
    feature_df = pd.DataFrame()

    feature_df[str(data_type)+"_AC_coordinate_cc1"] = X_features_cc1
    feature_df[str(data_type)+"_AC_coordinate_cc2"] = X_features_cc2
    feature_df[str(data_type)+"_AC_coordinate_cc3"] = X_features_cc3
    feature_df[str(data_type)+"_AC_coordinate_cc4"] = X_features_cc4
    feature_df["Label"] = label

    # Subject
    feature_df["Subject"] = subject

    # Training or test set (boolean)
    feature_df["Train"] = train

    return feature_df

In [68]:
features_dataframes = {}


for label in label_list:
    
    # Train data Adcock Carlson coordinates 
    coord_1, coord_2, coord_3, coord_4 = carlsson_coordinates(persistence_diagrams["Label_"+str(label)])
    features_dataframes[label] = create_dataframe(data_type, coord_1, coord_2, coord_3, coord_4, \
                                                        label, True)

In [69]:
# Concatenate and save dataframes
feature_df = pd.concat([features_dataframes[1], features_dataframes[2], features_dataframes[3], \
                              features_dataframes[4], features_dataframes[5], features_dataframes[7]], ignore_index=True)

feature_df.to_csv("Features/"+str(subject)+"/"+str(data_type)+"/Advanced_Features.csv")